In [ ]:
import pandas as pd

news_original = pd.read_csv('./news.csv', index_col=0)
news_original.head()

,authors,title,publish_date,description,text,url
0,[],'More vital now:' Gay-straight alliances go vi...,2020-05-03 1:30,Lily Overacker and Laurell Pallot start each g...,Lily Overacker and Laurell Pallot start each g...,https://www.cbc.ca/news/canada/calgary/gay-str...
1,[],Scientists aim to 'see' invisible transmission...,2020-05-02 8:00,Some researchers aim to learn more about how t...,"This is an excerpt from Second Opinion, a week...",https://www.cbc.ca/news/technology/droplet-tra...
2,['The Canadian Press'],Coronavirus: What's happening in Canada and ar...,2020-05-02 11:28,Canada's chief public health officer struck an...,The latest: The lives behind the numbers: Wha...,https://www.cbc.ca/news/canada/coronavirus-cov...
3,[],"B.C. announces 26 new coronavirus cases, new c...",2020-05-02 18:45,B.C. provincial health officer Dr. Bonnie Henr...,B.C. provincial health officer Dr. Bonnie Henr...,https://www.cbc.ca/news/canada/british-columbi...
4,[],"B.C. announces 26 new coronavirus cases, new c...",2020-05-02 18:45,B.C. provincial health officer Dr. Bonnie Henr...,B.C. provincial health officer Dr. Bonnie Henr...,https://www.cbc.ca/news/canada/british-columbi...


In [ ]:
news = news_original.copy()

contents = []
for index, row in news.iterrows():
  content = ''
  if isinstance(row['title'], str):
    content += row['title']
  if isinstance(row['description'], str):
    content += row['description']
  if isinstance(row['text'], str):
    content += row['text']
  contents.append(content)

news['content'] = contents[:]

del news['title'], news['description'], news['text']
news.head()

,authors,publish_date,url,content
0,[],2020-05-03 1:30,https://www.cbc.ca/news/canada/calgary/gay-str...,'More vital now:' Gay-straight alliances go vi...
1,[],2020-05-02 8:00,https://www.cbc.ca/news/technology/droplet-tra...,Scientists aim to 'see' invisible transmission...
2,['The Canadian Press'],2020-05-02 11:28,https://www.cbc.ca/news/canada/coronavirus-cov...,Coronavirus: What's happening in Canada and ar...
3,[],2020-05-02 18:45,https://www.cbc.ca/news/canada/british-columbi...,"B.C. announces 26 new coronavirus cases, new c..."
4,[],2020-05-02 18:45,https://www.cbc.ca/news/canada/british-columbi...,"B.C. announces 26 new coronavirus cases, new c..."


Pre-processing


In [ ]:
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

from nltk import download
download('wordnet')
download('averaged_perceptron_tagger')

from nltk import WordNetLemmatizer, pos_tag
from nltk.corpus import wordnet

def get_pos(token):  # get part of speech
  tag = pos_tag([token])[0][1][0].upper()
  tag_dict = {
    "J": wordnet.ADJ,
    "N": wordnet.NOUN,
    "V": wordnet.VERB,
    "R": wordnet.ADV
  }
  return tag_dict.get(tag, wordnet.NOUN)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
wnl = WordNetLemmatizer()

preprocessed_content = []

for index, row in news.iterrows():
  content = []
  for token in simple_preprocess(row['content']):
    if not (token in STOPWORDS):
      content.append(wnl.lemmatize(token, pos=get_pos(token)))
  #content = [ wnl.lemmatize(token, pos=get_pos(token)) for token in simple_preprocess(row['content']) if not (token in STOPWORDS)]
  preprocessed_content.append(' '.join(content))

news['preprocessed_content'] = preprocessed_content[:]
news

,authors,publish_date,url,content,preprocessed_content
0,[],2020-05-03 1:30,https://www.cbc.ca/news/canada/calgary/gay-str...,'More vital now:' Gay-straight alliances go vi...,vital gay straight alliance virtual coronaviru...
1,[],2020-05-02 8:00,https://www.cbc.ca/news/technology/droplet-tra...,Scientists aim to 'see' invisible transmission...,scientist aim invisible transmission coronavir...
2,['The Canadian Press'],2020-05-02 11:28,https://www.cbc.ca/news/canada/coronavirus-cov...,Coronavirus: What's happening in Canada and ar...,coronavirus happen canada world canada chief p...
3,[],2020-05-02 18:45,https://www.cbc.ca/news/canada/british-columbi...,"B.C. announces 26 new coronavirus cases, new c...",announces new coronavirus case new community o...
4,[],2020-05-02 18:45,https://www.cbc.ca/news/canada/british-columbi...,"B.C. announces 26 new coronavirus cases, new c...",announces new coronavirus case new community o...
...,...,...,...,...,...
6782,['Cbc News'],2020-04-06 10:49,https://www.cbc.ca/news/canada/windsor/fishing...,Fishing industry in Windsor-Essex shut down du...,fishing industry windsor essex shut peak seaso...
6783,['Cbc News'],2020-03-18 17:08,https://www.cbc.ca/news/canada/ottawa/tunneys-...,Health Canada worker tests positive for COVID-...,health canada worker test positive covid healt...
6784,['Cbc News'],2020-04-19 11:30,https://www.cbc.ca/news/canada/newfoundland-la...,Taxis adapting to COVID-19 with plastic shield...,taxi adapt covid plastic shield passenger limi...
6785,[],2020-03-31 12:55,https://www.cbc.ca/news/canada/edmonton/shawn-...,'An amazing man': Young father identified as n...,amaze man young father identify northern alber...


In [ ]:
from gensim.models import Phrases
from gensim.models.phrases import Phraser

bigram_model = Phrases([ content.split() for content in preprocessed_content ])

In [ ]:
enriched_content = []

for index, row in news.iterrows():
  enriched_content.append(bigram_model[row['preprocessed_content'].split()])

news['preprocessed_content'] = [ ' '.join(content) for content in enriched_content ]
news.head()

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


,authors,publish_date,url,content,preprocessed_content
0,[],2020-05-03 1:30,https://www.cbc.ca/news/canada/calgary/gay-str...,'More vital now:' Gay-straight alliances go vi...,vital gay_straight alliance virtual coronaviru...
1,[],2020-05-02 8:00,https://www.cbc.ca/news/technology/droplet-tra...,Scientists aim to 'see' invisible transmission...,scientist aim invisible transmission coronavir...
2,['The Canadian Press'],2020-05-02 11:28,https://www.cbc.ca/news/canada/coronavirus-cov...,Coronavirus: What's happening in Canada and ar...,coronavirus happen canada world canada chief_p...
3,[],2020-05-02 18:45,https://www.cbc.ca/news/canada/british-columbi...,"B.C. announces 26 new coronavirus cases, new c...",announces_new coronavirus case new community o...
4,[],2020-05-02 18:45,https://www.cbc.ca/news/canada/british-columbi...,"B.C. announces 26 new coronavirus cases, new c...",announces_new coronavirus case new community o...


Topic Modeling

In [ ]:
from gensim.corpora import Dictionary

dictionary = Dictionary(enriched_content)
#bag of words
bow = [ dictionary.doc2bow(content) for content in enriched_content ]

In [8]:
from gensim.models  import LdaMulticore

model = LdaMulticore(
    bow,
    id2word = dictionary,
    num_topics = 25,
    chunksize = 1000,
    passes = 200,
    workers = 1,
    random_state = 10
)

In [10]:
model

In [9]:
for topic, words in model.print_topics(-1):
  print('Topic', topic)
  for word, weight in words:
    print(word, '%.2f'%weight)

Topic 0


ValueError: ignored